In [1]:
import jax
import jax.numpy as jnp
import numpy as np
import optax
from flax import linen as nn
from flax.training import train_state
from tqdm import tqdm
import pickle
import matplotlib.pyplot as plt

import jaxmetric as jm

In [2]:
key = jax.random.PRNGKey(0)

psi=0

poly = lambda x: x[0]**5 + x[1]**5 + x[2]**5 +x[3]**5 + x[4]**5 + psi* x[0]*x[1]*x[2]*x[3]*x[4]

points = np.array([[0.5457718 +0.42877685j, -0.96667378+0.00622714j, -0.2195527-0.23682649j, 0.01382411+0.10245113j, 1.+0.j]])        

In [8]:
points = jm.point_gen.generate_points_calabi_yau(key, (4,),poly,10)

In [9]:
met = jm.metrics.get_2form_FS_proj_prod((4,),[1],points)[0]
print(jnp.linalg.eigvals(met))
print(met - met.T.conj())

[ 3.2506463e-01-3.0967284e-11j -7.0743509e-09-7.7495337e-12j
  3.2506472e-01+8.3819032e-09j  3.2506463e-01+1.4901161e-08j
  3.2506469e-01+0.0000000e+00j]
[[0.-1.9225973e-11j 0.-1.1641532e-10j 0.+4.6566129e-10j 0.+0.0000000e+00j
  0.+1.8626451e-09j]
 [0.-1.1641532e-10j 0.+1.5458940e-09j 0.-4.6566129e-10j 0.+0.0000000e+00j
  0.+0.0000000e+00j]
 [0.+4.6566129e-10j 0.-4.6566129e-10j 0.+1.8279149e-09j 0.+0.0000000e+00j
  0.+0.0000000e+00j]
 [0.+0.0000000e+00j 0.+0.0000000e+00j 0.+0.0000000e+00j 0.+0.0000000e+00j
  0.+0.0000000e+00j]
 [0.+1.8626451e-09j 0.+0.0000000e+00j 0.+0.0000000e+00j 0.+0.0000000e+00j
  0.-1.4065049e-09j]]


In [15]:
met = jm.metrics.get_ref_metric((4,),[1],poly,points)[0]
print(jnp.linalg.eigvals(met))
print(jnp.abs(met - met.T.conj()))
print(jnp.real(met - met.T.conj()))
print(jnp.imag(met - met.T.conj()))

[0.24773018+0.01658738j 0.37021127-0.04031334j 0.41854146-0.02421714j]
[[0.04212835 0.02084592 0.00313691]
 [0.02084592 0.07114129 0.01839866]
 [0.00313691 0.01839866 0.06687333]]
[[ 0.         -0.01822909 -0.00231864]
 [ 0.01822909  0.          0.01743348]
 [ 0.00231864 -0.01743348  0.        ]]
[[ 0.04212835 -0.01011201  0.00211284]
 [-0.01011201 -0.07114129  0.00588086]
 [ 0.00211284  0.00588086 -0.06687333]]


In [6]:
pb = jm.pullback.get_pullback(jnp.array([points[0]]),[4],poly)[0]

In [7]:
pb

Array([[ 1.00000000e+00+0.0000000e+00j, -2.39019617e-01+1.1609333e-01j,
         0.00000000e+00+0.0000000e+00j,  0.00000000e+00+0.0000000e+00j,
         0.00000000e+00+0.0000000e+00j],
       [ 0.00000000e+00+0.0000000e+00j, -1.22598419e-02-2.1941222e-03j,
         1.00000000e+00+0.0000000e+00j,  0.00000000e+00+0.0000000e+00j,
         0.00000000e+00+0.0000000e+00j],
       [ 0.00000000e+00+0.0000000e+00j,  1.14102106e-04-6.3933039e-05j,
         0.00000000e+00+0.0000000e+00j,  1.00000000e+00+0.0000000e+00j,
         0.00000000e+00+0.0000000e+00j]], dtype=complex64)